# Action Recognition with MoViNet

This notebook demonstrates how to use the MoViNet model for action recognition in videos. We will go through the following steps:

1. Setup Environment
2. Import Libraries
3. Load Pre-trained MoViNet Model
4. Preprocess Video Frames
5. Initialize Model States
6. Run Inference Frame-by-Frame
7. Map Predictions to Actions

## 1. Setup Environment

First, ensure you have the necessary libraries installed. You can install them using pip:

In [1]:
!pip install remotezip tqdm opencv-python opencv-python-headless tf-models-official

## 2. Import Libraries

In [2]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official)
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

## 3. Load Pre-trained MoViNet Model

In [3]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

## 4. Preprocess Video Frames

In [4]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (resolution, resolution))
        frames.append(frame)
    cap.release()
    return np.array(frames)

video_path = 'path_to_video.mp4'  # Replace with your video path
frames = extract_frames(video_path)
frames = frames / 255.0  # Normalize frames

## 5. Initialize Model States

In [5]:
initial_state = model.init_states(frames[tf.newaxis, ...].shape)

## 6. Run Inference Frame-by-Frame

In [6]:
state = initial_state.copy()
all_logits = []

for n in range(len(frames)):
    inputs = state
    inputs['image'] = frames[tf.newaxis, n:n+1, ...]
    outputs = model(inputs)
    
    # Check if the model returns a tuple or a single output
    if isinstance(outputs, tuple):
        logits, state = outputs
    else:
        logits = outputs
        state = None  # If the model does not return state, set it to None
    
    all_logits.append(logits)

probabilities = tf.nn.softmax(all_logits, axis=-1)


## 7. Map Predictions to Actions

In [7]:
def get_top_k(probs, k=5):
    top_k_indices = np.argsort(probs)[::-1][:k]
    return [(i, probs[i]) for i in top_k_indices]

actions = [
    'abseiling', 'acting in play', 'adjusting glasses', 'air drumming', 'alligator wrestling',
    'answering questions', 'applauding', 'applying cream', 'archaeological excavation', 'archery',
    'arguing', 'arm wrestling', 'arranging flowers', 'assembling bicycle', 'assembling computer',
    'attending conference', 'auctioning', 'backflip (human)', 'baking cookies', 'bandaging',
    'barbequing', 'bartending', 'base jumping', 'bathing dog', 'battle rope training',
    'beatboxing', 'bee keeping', 'belly dancing', 'bench pressing', 'bending back',
    'bending metal', 'biking through snow', 'blasting sand', 'blowdrying hair', 'blowing bubble gum',
    'blowing glass', 'blowing leaves', 'blowing nose', 'blowing out candles', 'bobsledding',
    'bodysurfing', 'bookbinding', 'bottling', 'bouncing on bouncy castle', 'bouncing on trampoline',
    'bowling', 'braiding hair', 'breading or breadcrumbing', 'breakdancing', 'breaking boards',
    'breathing fire', 'brush painting', 'brushing hair', 'brushing teeth', 'building cabinet',
    'building lego', 'building sandcastle', 'building shed', 'bull fighting', 'bulldozing',
    'bungee jumping', 'burping', 'busking', 'calculating', 'calligraphy',
    'canoeing or kayaking', 'capoeira', 'capsizing', 'card stacking', 'card throwing',
    'carrying baby', 'cartwheeling', 'carving ice', 'carving pumpkin', 'casting fishing line',
    'catching fish', 'catching or throwing baseball', 'catching or throwing frisbee', 'catching or throwing softball',
    'celebrating', 'changing gear in car', 'changing oil', 'changing wheel (not on bike)', 'checking tires',
    'cheerleading', 'chewing gum', 'chiseling stone', 'chiseling wood', 'chopping meat',
    'chopping vegetables', 'chopping wood', 'clam digging', 'clapping', 'clay pottery making',
    'clean and jerk', 'cleaning gutters', 'cleaning pool', 'cleaning shoes', 'cleaning toilet',
    'cleaning windows', 'climbing a rope', 'climbing ladder', 'climbing tree', 'coloring in',
    'combing hair', 'contact juggling', 'contorting', 'cooking egg', 'cooking on campfire',
    'cooking sausages (not on barbeque)', 'cooking scallops', 'cosplaying', 'counting money', 'country line dancing',
    'cracking back', 'cracking knuckles', 'cracking neck', 'crawling baby', 'crossing eyes',
    'crossing river', 'crying', 'cumbia', 'curling (sport)', 'curling hair',
    'cutting apple', 'cutting nails', 'cutting orange', 'cutting pineapple', 'cutting watermelon',
    'dancing ballet', 'dancing charleston', 'dancing gangnam style', 'dancing macarena', 'deadlifting',
    'decorating the christmas tree', 'delivering mail', 'dining', 'directing traffic', 'disc golfing',
    'diving cliff', 'docking boat', 'dodgeball', 'doing aerobics', 'doing jigsaw puzzle',
    'doing laundry', 'doing nails', 'drawing', 'dribbling basketball', 'drinking shots',
    'driving car', 'driving tractor', 'drooling', 'drop kicking', 'drumming fingers',
    'dumpster diving', 'dunking basketball', 'dyeing eyebrows', 'dyeing hair', 'eating burger',
    'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts', 'eating hotdog',
    'eating ice cream', 'eating spaghetti', 'eating watermelon', 'egg hunting', 'embroidering',
    'exercising with an exercise ball', 'extinguishing fire', 'faceplanting', 'falling off bike', 'falling off chair',
    'feeding birds', 'feeding fish', 'feeding goats', 'fencing (sport)', 'fidgeting',
    'finger snapping', 'fixing bicycle', 'fixing hair', 'flint knapping', 'flipping pancake',
    'fly tying', 'flying kite', 'folding clothes', 'folding napkins', 'folding paper',
    'front raises', 'frying vegetables', 'geocaching', 'getting a haircut', 'getting a piercing',
    'getting a tattoo', 'giving or receiving award', 'gold panning', 'golf chipping', 'golf driving',
    'golf putting', 'gospel singing in church', 'grinding meat', 'grooming dog', 'grooming horse',
    'gymnastics tumbling', 'hammer throw', 'hand washing clothes', 'head stand', 'headbanging',
    'headbutting', 'high jump', 'high kick', 'historical reenactment', 'hitting baseball',
    'hockey stop', 'holding snake', 'home roasting coffee', 'hopscotch', 'hoverboarding',
    'huddling', 'hugging (not baby)', 'hugging baby', 'hula hooping', 'hurdling',
    'hurling (sport)', 'ice climbing', 'ice fishing', 'ice skating', 'ice swimming',
    'inflating balloons', 'installing carpet', 'ironing', 'ironing hair', 'javelin throw',
    'jaywalking', 'jetskiing', 'jogging', 'juggling balls', 'juggling fire',
    'juggling soccer ball', 'jumping bicycle', 'jumping into pool', 'jumping jacks', 'jumpstyle dancing',
    'karaoke', 'kicking field goal', 'kicking soccer ball', 'kissing', 'kitesurfing',
    'knitting', 'krumping', 'land sailing', 'laughing', 'lawn mower racing',
    'laying bricks', 'laying concrete', 'laying stone', 'laying tiles', 'leatherworking',
    'licking', 'lifting hat', 'lighting fire', 'lock picking', 'long jump',
    'longboarding', 'looking at phone', 'luge', 'lunge', 'making a cake',
    'making a sandwich', 'making balloon shapes', 'making bubbles', 'making cheese', 'making horseshoes',
    'making jewelry', 'making paper aeroplanes', 'making pizza', 'making snowman', 'making sushi',
    'making tea', 'making the bed', 'marching', 'marriage proposal', 'massaging back',
    'massaging feet', 'massaging legs', 'massaging neck', 'massaging person head', 'milking cow',
    'moon walking', 'mopping floor', 'mosh pit dancing', 'motorcycling', 'mountain climber (exercise)',
    'moving furniture', 'mowing lawn', 'mushroom foraging', 'needle felting', 'news anchoring',
    'opening bottle (not wine)', 'opening door', 'opening present', 'opening refrigerator', 'opening wine bottle',
    'packing', 'paragliding', 'parasailing', 'parkour', 'passing American football (in game)',
    'passing american football (not in game)', 'passing soccer ball', 'peeling apples', 'peeling potatoes',
    'person collecting garbage', 'petting animal (not cat)', 'petting cat', 'photobombing', 'photocopying',
    'picking fruit', 'pillow fight', 'pinching', 'pirouetting', 'planing wood',
    'planting trees', 'plastering', 'playing accordion', 'playing badminton', 'playing bagpipes',
    'playing basketball', 'playing bass guitar', 'playing beer pong', 'playing blackjack', 'playing cello',
    'playing chess', 'playing clarinet', 'playing controller', 'playing cricket', 'playing cymbals',
    'playing darts', 'playing didgeridoo', 'playing dominoes', 'playing drums', 'playing field hockey',
    'playing flute', 'playing gong', 'playing guitar', 'playing hand clapping games', 'playing harmonica',
    'playing harp', 'playing ice hockey', 'playing keyboard', 'playing kickball', 'playing laser tag',
    'playing lute', 'playing maracas', 'playing marbles', 'playing monopoly', 'playing netball',
    'playing ocarina', 'playing organ', 'playing paintball', 'playing pan pipes', 'playing piano',
    'playing pinball', 'playing ping pong', 'playing poker', 'playing polo', 'playing recorder',
    'playing rubiks cube', 'playing saxophone', 'playing scrabble', 'playing squash or racquetball', 'playing tennis',
    'playing trombone', 'playing trumpet', 'playing ukulele', 'playing violin', 'playing volleyball',
    'playing with trains', 'playing xylophone', 'poking bellybutton', 'pole vault', 'polishing metal',
    'popping balloons', 'pouring beer', 'preparing salad', 'presenting weather forecast', 'pull ups',
    'pumping fist', 'pumping gas', 'punching bag', 'punching person (boxing)', 'push up',
    'pushing car', 'pushing cart', 'pushing wheelbarrow', 'pushing wheelchair', 'putting in contact lenses',
    'putting on eyeliner', 'putting on foundation', 'putting on lipstick', 'putting on mascara', 'putting on sari',
    'putting on shoes', 'raising eyebrows', 'reading book', 'reading newspaper', 'recording music',
    'repairing puncture', 'riding a bike', 'riding camel', 'riding elephant', 'riding mechanical bull',
    'riding mule', 'riding or walking with horse', 'riding scooter', 'riding snow blower', 'riding unicycle',
    'ripping paper', 'roasting marshmallows', 'roasting pig', 'robot dancing', 'rock climbing',
    'rock scissors paper', 'roller skating', 'rolling pastry', 'rope pushdown', 'running on treadmill',
    'sailing', 'salsa dancing', 'sanding floor', 'sausage making', 'sawing wood',
    'scrambling eggs', 'scrapbooking', 'scrubbing face', 'scuba diving', 'separating eggs',
    'setting table', 'sewing', 'shaking hands', 'shaking head', 'shaping bread dough',
    'sharpening knives', 'sharpening pencil', 'shaving head', 'shaving legs', 'shearing sheep',
    'shining flashlight', 'shining shoes', 'shooting basketball', 'shooting goal (soccer)', 'shopping',
    'shot put', 'shoveling snow', 'shucking oysters', 'shuffling cards', 'shuffling feet',
    'side kick', 'sign language interpreting', 'singing', 'sipping cup', 'situp',
    'skateboarding', 'ski jumping', 'skiing crosscountry', 'skiing mono', 'skiing slalom',
    'skipping rope', 'skipping stone', 'skydiving', 'slacklining', 'slapping',
    'sled dog racing', 'sleeping', 'smashing', 'smelling feet', 'smoking',
    'smoking hookah', 'smoking pipe', 'snatch weight lifting', 'sneezing', 'snorkeling',
    'snowboarding', 'snowkiting', 'snowmobiling', 'somersaulting', 'spelunking',
    'spinning poi', 'spray painting', 'springboard diving', 'square dancing', 'squat',
    'standing on hands', 'staring', 'steer roping', 'sticking tongue out', 'stomping grapes',
    'stretching arm', 'stretching leg', 'sucking lolly', 'surfing crowd', 'surfing water',
    'sweeping floor', 'swimming backstroke', 'swimming breast stroke', 'swimming butterfly stroke', 'swimming front crawl',
    'swing dancing', 'swinging baseball bat', 'swinging on something', 'sword fighting', 'sword swallowing',
    'tackling', 'tagging graffiti', 'tai chi', 'talking on cell phone', 'tango dancing',
    'tap dancing', 'tapping guitar', 'tapping pen', 'tasting beer', 'tasting food',
    'tasting wine', 'testifying', 'texting', 'threading needle', 'throwing axe',
    'throwing ball (not baseball or American football)', 'throwing discus', 'throwing knife', 'throwing snowballs', 'throwing tantrum',
    'throwing water balloon', 'tickling', 'tie dying', 'tightrope walking', 'tiptoeing',
    'tobogganing', 'tossing coin', 'training dog', 'trapezing', 'trimming or shaving beard',
    'trimming shrubs', 'trimming trees', 'triple jump', 'twiddling fingers', 'tying bow tie',
    'tying knot (not on a tie)', 'tying necktie', 'tying shoe laces', 'unboxing', 'unloading truck',
    'using a microscope', 'using a paint roller', 'using a power drill', 'using a sledge hammer', 'using a wrench',
    'using atm', 'using bagging machine', 'using circular saw', 'using inhaler', 'using puppets',
    'using remote controller (not gaming)', 'using segway', 'vacuuming floor', 'visiting the zoo', 'wading through mud',
    'wading through water', 'waiting in line', 'waking up', 'walking the dog', 'walking through snow',
    'washing dishes', 'washing feet', 'washing hair', 'washing hands', 'watching tv',
    'water skiing', 'water sliding', 'watering plants', 'waving hand', 'waxing back',
    'waxing chest', 'waxing eyebrows', 'waxing legs', 'weaving basket', 'weaving fabric',
    'welding', 'whistling', 'windsurfing', 'winking', 'wood burning (art)',
    'wrapping present', 'wrestling', 'writing', 'yarn spinning', 'yawning',
    'yoga', 'zumba']

for i, logits in enumerate(all_logits):
    probs = tf.nn.softmax(logits[0], axis=-1)
    top_k = get_top_k(probs)
    print(f"Frame {i}:")
    for label, p in top_k:
        print(f'{actions[label]:20s}: {p:.3f}')
    print()